In [1]:
import numpy as np

np.vander([2],3)

array([[4, 2, 1]])

In [2]:
## Old Poly Produc

# def polyProduct(x, y, xcoeffs:'list[float]'=None, ycoeffs:'list[float]'=None,*coeffs):
#         coeff_len = len(coeffs)
#         return XLin(x, *xcoeffs) * YLin(y, ycoeffs) + sum([i for i in range(coeff_len)])

In [3]:
import inspect

In [4]:
def func(a,c,b,d,*args):
    return a+b+c+d



In [5]:
func_sig=inspect.signature(func)

In [18]:
[func_sig.parameters.get(i) for i in func_sig.parameters if i!='b']

NameError: name 'func_sig' is not defined

In [ ]:


import inspect
import numpy as np
import scipy.optimize as optimist
import numpy.linalg as linalg



# A polynomial function maker and an XY Product Maker
def functionMaker(n: int, m: int, dx: int = 1, dy: int = 1):
    '''
    Args:
        n: int - The size of the polynomial in the Minor axis
        m: int - The size of the polynomial in the Major axis 
    Returns:
        XLin: function - A linear Polynomial function that depends soley on x
        YLin: function - A linear Polynomial function that depends soley on y
        polyProduct: function - A linear Polynomial function that depends on x and y
    Details:
    
    '''
    print(n,m)
    
    # Defines an linear Polynomial function dependent only on x given n Parameters
    def XLin(x, *args):
        '''
        XLin: A linear polynomial in one axis.
        
        We generalise the idea of unkowns.
        
            x:  The value of x at the point we want to soove the polynomial
        *args:  The polynomial coefficients in order of most important to least important 
        '''
        
        # initilizes the results variable to zero
        result = np.float64(0.0)
        x = np.float64(x)
        print(args)
        # Evaluates each polynomial terms and adds them together
        for i in range(1, n+1):
            result = result+args[-i]*np.power(x,(i))
            print(f"{args[-i]}*{x}^({i})= {args[-i]}*{x**i} ={result+args[-i]*x**(i)}")
            
        print(result)
        
        # constrain to a widely availible floating point representation
        return result

    # Defines an linear Polynomial function for X given n Parameters
    # Does the same thing that the XLin does
    def YLin(y, *args):
         
        result = 0
        
        for i in range(1, m + dy, dy):
            result += args[-i] * (y ** (i))
            
        return np.float64(result)

    

    # Everything in this function context beyond here is for compatibility with 
    # introspective curve fitter. For most people this can be ignored
    
    xParam = [inspect.Parameter(
                f"x",
                inspect._ParameterKind.POSITIONAL_OR_KEYWORD,
                default=0,
                annotation=np.float64,
            )]

    xCoeffParams = [
            inspect.Parameter(
                f"x_{i}",
                inspect._ParameterKind.POSITIONAL_OR_KEYWORD,
                default=0,
                annotation=np.float64,
            )
            for i in range(0, n, 1)
        ]
    
    xParams  = xParam+xCoeffParams
    
    XLin.__signature__ = inspect.Signature(
        xParams,
        return_annotation=np.float64,
        __validate_parameters__=True,
    )
    
    yParam = [
            inspect.Parameter(
                f"y",
                inspect._ParameterKind.POSITIONAL_OR_KEYWORD,
                default=0,
                annotation=np.float64,
            )]
    
    yCoeffParams = [
            inspect.Parameter(
                f"y_{i}",
                inspect._ParameterKind.POSITIONAL_OR_KEYWORD,
                default=0,
                annotation=np.float64,
            )
            for i in range(0, m, 1)
        ]
    
    yParams = yParam+yCoeffParams
    
    YLin.__signature__ = inspect.Signature(
        yParams,
        return_annotation=np.float64,
        __validate_parameters__=True,
    )
    
    
    
    
    
    
    
    mixedParam = [
        inspect.Parameter(
                f"x",
                inspect._ParameterKind.POSITIONAL_OR_KEYWORD,
                default=0,
                annotation=np.float64,
            ),
        inspect.Parameter(
                f"y",
                inspect._ParameterKind.POSITIONAL_OR_KEYWORD,
                default=0,
                annotation=np.float64,
            ),
        inspect.Parameter(
                f"xcoeffs",
                inspect._ParameterKind.POSITIONAL_OR_KEYWORD,
                default=0,
                annotation=np.float64,
            ),
        inspect.Parameter(
                f"ycoeffs",
                inspect._ParameterKind.POSITIONAL_OR_KEYWORD,
                default=0,
                annotation=np.float64,
            ),
    ]
    
    mixedParams = [
            inspect.Parameter(
                f"c_{i}",
                inspect._ParameterKind.POSITIONAL_OR_KEYWORD,
                default=0,
                annotation=np.float64,
            )
            for i in range(0, n*m, 1)
        ]
    
    def polyProduct(x, y, xcoeffs:'list[float]'=None, ycoeffs:'list[float]'=None,*coeffs):
        coeff_len = len(coeffs)
        interpolated_func = XLin(x, *xcoeffs) * YLin(y, ycoeffs)
        
        xs=np.vander([x],n)
        ys=np.vander([y],m)
        fixingProduct = ((xs.T@ys).flatten())*coeffs
        
        return interpolated_func+fixingProduct
            

    
    polyProduct.__signature__ = inspect.Signature(
        mixedParams,
        return_annotation=np.float64,
        __validate_parameters__=True,
    )
    
    
    
    # return the X-dependent Linear polynomial, 
    # Y Linear Polynomial and Polynomial Product function
    return XLin, YLin, polyProduct


# A default ready made but slow fitting function 
def InterpolateGrid(Grid:'np.ndarray',x0:'np.ndarray',y0:'np.ndarray',x1:'np.ndarray',y1:'np.ndarray',dy:float=1.0,dx:float=1.0):
    (n,m) = Grid.shape
    XPolyNomial,YPolyNomial,XYPolyNomial=functionMaker(n,m)
    
    Xs=np.arange(x0,x1+dx,dx)
    Ys=np.arange(y0,y1+dy,dy)
    
    xCov=None
    yCov = None
    
    print(XPolyNomial.__signature__)
    print(YPolyNomial.__signature__)
    try:
        # Using curve fit is lazy but its better than writing a lsq function
        xOptimal,xCov=optimist.curve_fit(XPolyNomial,Xs,Grid[0,:])
        
    except RuntimeError:
        xOptimal = np.polyfit(Xs,Grid[0,:],Xs.shape[0])
    print('XOptimal=',xOptimal)
    print('XCov=',xCov) if xCov is not None else None
    try:
        yOptimal,yCov=optimist.curve_fit(YPolyNomial,Ys,Grid[:,0].T)
        
    except RuntimeError:
        yOptimal = np.polyfit(Ys,Grid[:,0].T,Ys.shape[0])

    print('YOptimal=',yOptimal)
    print('XCov=',xOptimal) if yCov is not None else None
    def __innerProduct(y:np.float64):
        nonlocal xOptimal,yOptimal
        def _innerProduct(x:np.float64,*args):
            nonlocal xOptimal,yOptimal
            return XYPolyNomial(x,y,xOptimal,yOptimal,*args)
        sig = inspect.signature(XYPolyNomial)
        sig = inspect.Signature([sig.parameters.get(param) for param in sig.parameters if param!='y'])
        
        _innerProduct.__signature__ = sig
        
        # TODO add a function signature to _innerProduct before returning it
        return _innerProduct
    XYoptimal = None
    XYcov = None
    for i in range(0,Ys.shape[0]):
        polyProd = __innerProduct(Ys[i])
        
        if (XYoptimal is not None):
            XYoptimal,XYcov = optimist.curve_fit(polyProd,Xs,Grid[i,:],p0=XYoptimal,sigma=XYcov,maxfev=999)
        else:
            XYoptimal,XYcov = optimist.curve_fit(polyProd,Xs,Grid[i,:],maxfev=999)
    return xOptimal,yOptimal,XYoptimal
        
                                      

In [38]:
XPolyNomial,YPolyNomial,XYPolyNomial=functionMaker(3,3)
XPolyNomial(0,1,2,3)

0**0

3 3
(1, 2, 3)
3*0^(1)= 3*0 =0.0
2*0^(2)= 2*0 =0.0
1*0^(3)= 1*0 =0.0
0.0


1

In [3]:
import numpy as np
G=np.array([
    [1,0,0],
    [0,1,0]
],np.float64)

Z=G.T

np.dot(G,Z)

array([[1., 0.],
       [0., 1.]])